In [1]:
#import packages here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb

%matplotlib inline


In [2]:
#import data here 

train_data = pd.read_csv('./train_noclean.csv')
test_data = pd.read_csv('./test_noclean.csv')


In [3]:

y_train = train_data.loc[:,'price']
train_data = train_data.drop(columns='price')
num_features = len(train_data.columns)
features = list(train_data.columns[2:num_features])
X_train = train_data.loc[:,features]

features = list(test_data.columns[2:num_features])

X_test = test_data.loc[:,features]
test_labels = test_data.loc[:,'id']
print(X_train.head(5))
print(X_train.shape)
print(X_test.shape)
print(y_train)
test_labels = test_labels.to_frame()

   minimum_nights  number_of_reviews  last_review  reviews_per_month  \
0               1                170          205               2.48   
1               1                 11          197               0.57   
2               3                  2         1911               0.08   
3               2                  1         1911               0.13   
4               2                 31         1912               1.54   

   calculated_host_listings_count  availability_365  host_since  \
0                               4               346          13   
1                               1                 0          14   
2                              19               360          17   
3                               1               267          19   
4                               5               365          15   

   host_is_superhost  bathrooms  bedrooms  ...  Da_hood_Villa Urquiza  \
0              False        1.0         0  ...                      0   
1              Fal

In [4]:
print(X_train.nunique())
print(X_train['host_since'].value_counts())

minimum_nights                     48
number_of_reviews                 248
last_review                        82
reviews_per_month                 504
calculated_host_listings_count     47
                                 ... 
Cancel_super_strict_30              2
Room_Entire home/apt                2
Room_Hotel room                     2
Room_Private room                   2
Room_Shared room                    2
Length: 69, dtype: int64
16    1550
15    1269
14    1217
18    1171
17    1085
19     988
13     888
12     792
11     481
10     138
20      79
9       22
8        1
Name: host_since, dtype: int64


In [5]:

dtest = xgb.DMatrix(X_test)

In [9]:
def run_model2(X_train, y_train, dtest, max_depth, n, eta, gamma):
    dtrain = xgb.DMatrix(X_train, label=y_train)
    param = {'max_depth':max_depth, 'objective':'multi:softmax', 'eta':eta, 'gamma':gamma, 'num_class':5, 'eval_metric':'merror'}
    num_round = n
    xgtrain = xgb.DMatrix(X_train, label=y_train)
    watchlist = [ (xgtrain,'train'),(dtest,'eval')]
    bst = xgb.train(param, xgtrain, num_round, watchlist,  early_stopping_rounds=10 , verbose_eval = False)
    return bst

In [10]:
def run_model(X_train, y_train, dtest, max_depth, n, eta, gamma):
    dtrain = xgb.DMatrix(X_train, label=y_train)
    param = {'max_depth':max_depth, 'objective':'multi:softmax', 'eta':eta, 'gamma':gamma, 'num_class':5}
    num_round = n
    bst = xgb.train(param, dtrain, num_round)
    return bst

In [11]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
accuracy = []
std = []
kf = KFold(n_splits = 10, shuffle=True)
#print(features)
y = np.concatenate([y_train], axis= 0)
for feature in features:
    X_train = train_data.loc[:,feature]
    X = np.concatenate([X_train], axis= 0)
    X = np.reshape(X, (len(X), 1))
    accuracy_feature = []
    for train, test in kf.split(X_train):
        X_train_kf, X_test_kf = X[train],X[test]
        
        y_train_kf, y_test_kf = y[train],y[test]
        
        #print(len(y_test_kf))
        #print(len(X_test_kf))
        #print(X_test_kf)
        
        dtest_kf = xgb.DMatrix(X_test_kf, label=y_test_kf)
        bst = run_model2(X_train_kf, y_train_kf, dtest_kf, 9, 300, 0.1, 1)
        
        ypred_test_kf = bst.predict(dtest_kf)
        
        accuracy_feature.append(accuracy_score(y_test_kf, ypred_test_kf))
    print(feature, np.mean(accuracy_feature))
    accuracy.append(np.mean(accuracy_feature))
    std.append(np.mean(accuracy_feature))
#bst = run_model(X_train, y_train, dtest, 9, 300, 0.1, 1)



minimum_nights 0.276726347346246
number_of_reviews 0.2556554853346297
last_review 0.2871595919794625
reviews_per_month 0.28282298782932047
calculated_host_listings_count 0.2925328787452345
availability_365 0.3130869986097963
host_since 0.31143495893355166
host_is_superhost 0.2739390101408114
bathrooms 0.34283522673967365
bedrooms 0.3689726564832109
beds 0.3516152589787546
bed_type 0.25452050763759176
cleaning_fee 0.44303362928468476
guests_included 0.31680717959215005
extra_people 0.3021373316616773
maximum_nights 0.2878832655289171
instant_bookable 0.2547277588721439
require_guest_profile_picture 0.2440871563936579
require_guest_phone_verification 0.24336294979061654
Da_hood_Agronomía 0.2547249869934925
Da_hood_Almagro 0.26681091096725773
Da_hood_Balvanera 0.26381642913798836
Da_hood_Barracas 0.2564823580584909
Da_hood_Belgrano 0.25782618615084135
Da_hood_Boedo 0.2562730812203089
Da_hood_Caballito 0.2588594572235158
Da_hood_Chacarita 0.25493309111378354
Da_hood_Coghlan 0.2546221076512

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

list_max_depth = list(np.arange(1, 10))

kf = KFold(n_splits = 10, shuffle=True)
kf_accuracy = []
depth_average = []
depth_std = []
X = np.concatenate([X_train], axis= 0)
y = np.concatenate([y_train], axis= 0)

for max_depth in list_max_depth: 
    print(max_depth)
    i=0
    accuracy_depth = []
    for train, test in kf.split(X_train):
        i+=1
        print(i)
        X_train_kf, X_test_kf = X[train],X[test]
        y_train_kf, y_test_kf = y[train],y[test]
        dtest_kf = xgb.DMatrix(X_test_kf, label=y_test_kf)
        bst = run_model(X_train_kf, y_train_kf, dtest_kf, max_depth, 500, 0.05, 3)
        
        ypred_test_kf = bst.predict(dtest_kf)
        
        accuracy_depth.append(accuracy_score(y_test_kf, ypred_test_kf))
    depth_average.append(np.mean(accuracy_depth))
    depth_std.append(np.std(accuracy_depth))
    print(np.mean(accuracy_depth))
    print(np.std(accuracy_depth))
    
    

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

list_max_depth = [10,15,20,30]

kf = KFold(n_splits = 5, shuffle=True)
kf_accuracy = []
depth_average = []
depth_std = []
X = np.concatenate([X_train], axis= 0)
y = np.concatenate([y_train], axis= 0)

for max_depth in list_max_depth: 
    print(max_depth)
    i=0
    accuracy_depth = []
    for train, test in kf.split(X_train):
        i+=1
        print(i)
        X_train_kf, X_test_kf = X[train],X[test]
        y_train_kf, y_test_kf = y[train],y[test]
        dtest_kf = xgb.DMatrix(X_test_kf, label=y_test_kf)
        bst = run_model(X_train_kf, y_train_kf, dtest_kf, max_depth, 500, 0.05, 3)
        
        ypred_test_kf = bst.predict(dtest_kf)
        
        accuracy_depth.append(accuracy_score(y_test_kf, ypred_test_kf))
    depth_average.append(np.mean(accuracy_depth))
    depth_std.append(np.std(accuracy_depth))
    print(np.mean(accuracy_depth))
    print(np.std(accuracy_depth))
    
    

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

eta_list = [0.01,0.03,0.05,0.08,0.1,0.15,0.2,0.3]

kf = KFold(n_splits = 5, shuffle=True)
kf_accuracy = []
eta_average = []
eta_std = []
X = np.concatenate([X_train], axis= 0)
y = np.concatenate([y_train], axis= 0)

for eta in eta_list: 
    eta_acc = []
    print(eta)
    i = 0
    for train, test in kf.split(X_train):
        i += 1
        print(i)
        X_train_kf, X_test_kf = X[train],X[test]
        y_train_kf, y_test_kf = y[train],y[test]
        dtest_kf = xgb.DMatrix(X_test_kf, label=y_test_kf)
        bst = run_model(X_train_kf, y_train_kf, dtest_kf, 9, 500, eta, 3)
        
        ypred_test_kf = bst.predict(dtest_kf)
        
        eta_acc.append(accuracy_score(y_test_kf, ypred_test_kf))
    eta_average.append(np.mean(eta_acc))
    eta_std.append(np.std(eta_acc))
    print(np.mean(eta_acc))
    print(np.std(eta_acc))
    
    

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

gamma_list = [0.1, 0.5, 1, 2, 3, 5, 10, 20]

kf = KFold(n_splits = 5, shuffle=True)
kf_accuracy = []
gamma_average = []
gamma_std = []
X = np.concatenate([X_train], axis= 0)
y = np.concatenate([y_train], axis= 0)

for gamma in gamma_list: 
    gamma_acc = []
    print(gamma)
    i = 0
    for train, test in kf.split(X_train):
        i += 1
        print(i)
        X_train_kf, X_test_kf = X[train],X[test]
        y_train_kf, y_test_kf = y[train],y[test]
        dtest_kf = xgb.DMatrix(X_test_kf, label=y_test_kf)
        bst = run_model(X_train_kf, y_train_kf, dtest_kf, 9, 500, 0.1, gamma)
        
        ypred_test_kf = bst.predict(dtest_kf)
        
        gamma_acc.append(accuracy_score(y_test_kf, ypred_test_kf))
    gamma_average.append(np.mean(gamma_acc))
    gamma_std.append(np.std(gamma_acc))
    
    
    

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

num_list = [50, 100, 150, 200, 300, 500, 1000]

kf = KFold(n_splits = 5, shuffle=True)
kf_accuracy = []
num_average = []
num_std = []
X = np.concatenate([X_train], axis= 0)
y = np.concatenate([y_train], axis= 0)

for num in num_list: 
    num_acc = []
    print(num)
    i = 0
    for train, test in kf.split(X_train):
        i += 1
        print(i)
        X_train_kf, X_test_kf = X[train],X[test]
        y_train_kf, y_test_kf = y[train],y[test]
        dtest_kf = xgb.DMatrix(X_test_kf, label=y_test_kf)
        bst = run_model(X_train_kf, y_train_kf, dtest_kf, 9, num, 0.1, 1)
        
        ypred_test_kf = bst.predict(dtest_kf)
        
        num_acc.append(accuracy_score(y_test_kf, ypred_test_kf))
    num_average.append(np.mean(num_acc))
    num_std.append(np.std(num_acc))
    print(np.mean(num_acc))
    print(np.std(num_acc))
    
    

In [ ]:
print(num_average)
print(num_std)

In [ ]:
print(gamma_average)
print(gamma_std)

In [ ]:
print(eta_average)
print(eta_std)

In [ ]:
print(depth_average)
print(depth_std)

In [ ]:
bst = run_model(X_train, y_train, dtest, 9, 300, 0.1, 1)

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
ypred_train = bst.predict(dtrain)
print(ypred_train)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train, ypred_train)

In [ ]:
ypred_test = bst.predict(dtest)

In [ ]:
plt.figure()
plt.hist(ypred_test)
plt.show()


In [ ]:
df = pd.DataFrame
test_labels['price'] = ypred_test
test_labels['price'] = test_labels['price'].map(lambda x: int(x))

In [ ]:
test_labels

In [ ]:
test_labels.to_csv(r'./submission.csv', index=False)